<a href="https://colab.research.google.com/github/Ignas12345/Magistro_projektas/blob/main/bandymas_klasifikuoti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [182]:
# Load data
url = 'https://raw.githubusercontent.com/Ignas12345/Magistro_projektas/refs/heads/main/counts_combined.csv'
df = pd.read_csv(url)
gene_names = df["miRNA_ID"].to_numpy()
df_trimmed = df.drop(df.columns[0], axis=1)
full_data = (df_trimmed.to_numpy(dtype=np.int32)).T

# Create labels: first 6 samples -> label 0, rest -> label 1
num_samples = full_data.shape[0]
labels = torch.zeros(num_samples, dtype=torch.float32)
labels[6:] = 1  # Set the rest of the samples to label 1

# Convert full_data to a PyTorch tensor
features = torch.tensor(full_data, dtype=torch.float32)

mean = features.mean(dim=0)
std = features.std(dim=0)
features = (features - mean) / (std + 1e-8)

# Combine features and labels into a dataset
dataset = TensorDataset(features, labels)

# Create a DataLoader for batching
batch_size = 1  # You can adjust this as needed
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example: Print the shape of a batch
for batch_features, batch_labels in data_loader:
    print(f"Batch features shape: {batch_features.shape}")
    print(f"Batch labels shape: {batch_labels.shape}")
    break


Batch features shape: torch.Size([1, 1881])
Batch labels shape: torch.Size([1])


In [162]:
for batch_features, batch_labels in data_loader:
    print(f"Batch features shape: {batch_features}")
    print(f"Batch labels shape: {batch_labels}")
    break

Batch features shape: tensor([[ 47204.,  46983.,  47453.,  ...,    408.,   2981., 137966.]])
Batch labels shape: tensor([1.])


In [179]:
import torch
import torch.nn as nn

class SparseNN(nn.Module):
    def __init__(self, input_dim):
        super(SparseNN, self).__init__()
        self.input_dim = input_dim

        self.first_layer_weights = nn.Parameter(torch.randn(input_dim))  # 1D tensor for scaling
        self.first_layer_biases = nn.Parameter(torch.randn(input_dim))  # 1D tensor for bias


        # Second layer: Sparse weighting (learnable parameters)
        self.output_weights_raw = nn.Parameter(torch.randn(input_dim))  # Raw weights

    def forward(self, x):
        # First layer: Linear transformation
        first_layer_output = x * self.first_layer_weights + self.first_layer_biases

        # Second layer: Apply sparse weights
        output_weights = torch.relu(self.output_weights_raw)  # Enforce non-negativity
        weighted_output = first_layer_output * output_weights  # Element-wise multiplication
  # Element-wise multiplication

        # Pooling: Sum the weighted outputs
        pooled_output = weighted_output.sum(dim=1)  # Summing across features

        # Final activation: tanh
        output = torch.sigmoid(pooled_output)
        return output



In [189]:
#initialize the model
input_dim = full_data.shape[1]
model = SparseNN(input_dim)

# Define a loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # L1 sparsity via weight_decay

In [196]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# L1 Regularization (sparsity enforcement)
def l1_regularization(weight, lambda_l1=0.001):
    return lambda_l1 * weight.abs().sum()

# Training Loop
num_epochs = 10  # Adjust based on dataset size and convergence
lambda_l1 = 0.001  # Regularization strength

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_features, batch_labels in data_loader:
        # Forward pass
        outputs = model(batch_features)  # Ensure shape compatibility for BCELoss
        # Compute binary cross-entropy loss
        loss = criterion(outputs, batch_labels)

        # Add L1 regularization for sparsity
        #loss += l1_regularization(model.output_weights_raw, lambda_l1)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Logging epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(data_loader):.4f}")

# Evaluation (after training)
model.eval()
with torch.no_grad():
    # Forward pass on the entire dataset
    predictions = model(features).squeeze()
    predictions = (predictions > 0.5).float()  # Convert probabilities to binary predictions

    # Compute accuracy
    accuracy = accuracy_score(labels.numpy(), predictions.numpy())
    print(f"Final Accuracy: {accuracy:.4f}")

Epoch [1/10], Loss: 2.8778
Epoch [2/10], Loss: 2.8777
Epoch [3/10], Loss: 2.8777
Epoch [4/10], Loss: 2.8777
Epoch [5/10], Loss: 2.8777
Epoch [6/10], Loss: 2.8777
Epoch [7/10], Loss: 2.8777
Epoch [8/10], Loss: 2.8777
Epoch [9/10], Loss: 2.8777
Epoch [10/10], Loss: 2.8777
Final Accuracy: 0.9712


In [195]:
print(features[0:2])
print(model(features).squeeze())

tensor([[2.0663, 2.0594, 2.0604,  ..., 2.2183, 0.9937, 1.1071],
        [5.3856, 5.3786, 5.4246,  ..., 5.9588, 4.3934, 2.3196]])
tensor([1.0000, 0.0020, 1.0000, 1.0000, 1.0000, 0.0019, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.9989, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,

In [39]:
# Print the weights of the first layer
print("First Layer Weights:")
print(torch.Tensor.size(model.first_layer_weights.data))

# Print the biases of the first layer (if present)
print("First Layer Biases:")
print(model.first_layer_biases.data)

# Print the weights of the second layer (output weights)
print("Second Layer (Sparse) Weights:")
print(torch.relu(model.output_weights_raw.data))

First Layer Weights:
torch.Size([1881])
First Layer Biases:
tensor([-0.8275, -1.5632, -0.5364,  ..., -1.3576, -0.1697, -0.6520])
Second Layer (Sparse) Weights:
tensor([0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.5769e+00, 0.0000e+00,
        1.7388e-04])
